In [2]:
import os
os.environ['BRT_ENV'] = 'prod'

import datetime
import json
import time

import boto3
import requests

#from brtdevkit.ml import Workflow
import brtdevkit
from brtdevkit.ml import KubeflowPipeline


In [3]:
# Find logfiels that match certain criteria
import time
#s3 = boto3.client('s3')

def get_logfile_directories(bucket, prefix, just_fsize =False):
    """
    Finds S3 directories with flatbuffer logfiles in them.
    :param bucket: S3 bucket to search in
    :param prefix: the prefix (pathname) underneath the bucket to be searched
    """

    s3 = boto3.client('s3')

    dir_list = []
    fsize = 0
    kwargs = {'Bucket': bucket, 'Prefix': prefix}
    while True:
        resp = s3.list_objects_v2(MaxKeys =10000, **kwargs)
        
        # This bit of code handles pagination in the boto library.
        if 'Contents' in resp:
            for entry in resp['Contents']:
                if entry['Key'].endswith('.bfbs'):
                    dir_list.append(os.path.dirname(entry['Key']))
                    fsize += entry['Size']
            try:
                kwargs['ContinuationToken'] = resp['NextContinuationToken']
            except KeyError:
                break
        else:
            break
    if just_fsize == True:
        return fsize
    else:
        return dir_list, fsize

In [3]:
# Get logfile names
d, f = get_logfile_directories('brt-dcm-data', 'db1')
d[0:5]

['db1/2/full/2021-05-06_21-50-02.523674239_210_2146.114868118',
 'db1/2/full/2021-05-06_21-50-02.523674239_210_2146.114868118/log_timestamps/2/aos/remote_timestamps/vpu0-0a/2/aos/aos-message_bridge-Timestamp',
 'db1/2/full/2021-05-06_21-50-02.523674239_210_2146.114868118/log_timestamps/2/aos/remote_timestamps/vpu0-0a/2/spray/brt-vpu-mapper-MapData',
 'db1/2/full/2021-05-06_21-50-02.523674239_210_2146.114868118/log_vpu0-0a_data/0/aos',
 'db1/2/full/2021-05-06_21-50-02.523674239_210_2146.114868118/log_vpu0-0a_data/lead']

In [5]:
import numpy as np
len(np.unique(d))

9568

In [4]:
regex = '2021-05-22_17-43-23'
regex2 = '2021-05-31_17-45-26'
paths_1 = [p for p in d if (p[17:36] == regex) or (p[17:36] == regex2)]#or (p[17:36] == regex2)
paths_2 = [p for p in d if (p[18:37] == regex) or (p[18:37] == regex2)]#or (p[18:37] == regex2)
paths = paths_1 + paths_2
in_db = [p.split('/log')[0] for p in paths]
in_db = [v for v in in_db  if (v.split('db1')[1][0:8] != '/vpu0-0a')]
run = list(set(in_db))
run = ['s3://brt-dcm-data/'+i for i in run]
run = sorted(run)
run

['s3://brt-dcm-data/db1/vpu0-10a/full/2021-05-22_17-43-23.105050338_321_10862.963027107',
 's3://brt-dcm-data/db1/vpu0-11a/full/2021-05-22_17-43-23.106521739_316_10863.938897702',
 's3://brt-dcm-data/db1/vpu0-12a/full/2021-05-22_17-43-23.088501270_321_10863.928497217',
 's3://brt-dcm-data/db1/vpu0-13a/full/2021-05-22_17-43-23.090155381_321_10880.115384940',
 's3://brt-dcm-data/db1/vpu0-14a/full/2021-05-22_17-43-23.091680542_321_10863.510682734',
 's3://brt-dcm-data/db1/vpu0-1a/full/2021-05-22_17-43-23.093221978_318_10865.410943602',
 's3://brt-dcm-data/db1/vpu0-2a/full/2021-05-22_17-43-23.095616081_318_10862.661160126',
 's3://brt-dcm-data/db1/vpu0-3a/full/2021-05-22_17-43-23.100048887_321_10865.041577876',
 's3://brt-dcm-data/db1/vpu0-4a/full/2021-05-22_17-43-23.103002524_318_10870.462574486']

In [13]:
print(len(run))
hr_1 = run[5:]
hr_2 = run[0:5]
print(len(hr_1))
print(len(hr_2))
hr_2

9
4
5


['s3://brt-dcm-data/db4/vpu0-10a/full/2021-06-25_16-45-22.471840230_376_3633.224988858',
 's3://brt-dcm-data/db4/vpu0-11a/full/2021-06-25_16-45-22.473659342_374_3633.251725291',
 's3://brt-dcm-data/db4/vpu0-12a/full/2021-06-25_16-45-22.477290429_378_3632.820045418',
 's3://brt-dcm-data/db4/vpu0-13a/full/2021-06-25_16-45-22.479998612_378_3632.749489819',
 's3://brt-dcm-data/db4/vpu0-14a/full/2021-06-25_16-45-22.481613193_377_3633.026604426']

In [16]:
paths_s3 = ['s3://brt-dcm-data/db1/vpu0-1a/full/2021-05-22_17-43-23.093221978_318_10865.410943602',
's3://brt-dcm-data/db1/vpu0-2a/full/2021-05-22_17-43-23.095616081_318_10862.661160126',
's3://brt-dcm-data/db1/vpu0-3a/full/2021-05-22_17-43-23.100048887_321_10865.041577876'
]

In [18]:
# For individual logs that couldn't be made into a multi-VPU spyglass
spyglass_wf = KubeflowPipeline.retrieve(id="9d83d431-e6f1-44a2-88e2-b7cc0db18407")

for r in paths_s3:
    name = 'wr_db1_' + r.split('2021-')[1][0:14] +r[26:28]
    print(name)
    spyglass_wf.submit(job_name=name, params={"s3_path": r})
    time.sleep(20)

wr_db1_05-22_17-43-23-1
wr_db1_05-22_17-43-23-2
wr_db1_05-22_17-43-23-3


In [7]:
path = ['s3://brt-dcm-data/db1/vpu0-1a/full/2021-05-22_17-43-23.093221978_318_10865.410943602',
's3://brt-dcm-data/db1/vpu0-2a/full/2021-05-22_17-43-23.095616081_318_10862.661160126',
's3://brt-dcm-data/db1/vpu0-3a/full/2021-05-22_17-43-23.100048887_321_10865.041577876']

In [8]:
spyglass_wf_big_mem = KubeflowPipeline.retrieve(name="Spyglass_big_mem")
name = 'wr_infield_05_22_17-43-23'
spyglass_wf_big_mem.submit(job_name=name, params={"s3_path": path})

{'created_at': 'Mon, 12 Jul 2021 16:35:58 GMT',
 'description': None,
 'error': None,
 'finished_at': 'Thu, 01 Jan 1970 00:00:00 GMT',
 'id': '1220af40-4d1a-4f47-a0b8-8389ef5c8afc',
 'metrics': None,
 'name': 'wr_infield_05_22_17-43-23',
 'pipeline_spec': {'parameters': [{'name': 's3_path',
    'value': '["s3://brt-dcm-data/db1/vpu0-1a/full/2021-05-22_17-43-23.093221978_318_10865.410943602", "s3://brt-dcm-data/db1/vpu0-2a/full/2021-05-22_17-43-23.095616081_318_10862.661160126", "s3://brt-dcm-data/db1/vpu0-3a/full/2021-05-22_17-43-23.100048887_321_10865.041577876"]'}],
  'pipeline_id': '38144d2c-baaa-491d-9434-224572b61fb8',
  'pipeline_manifest': None,
  'pipeline_name': 'Spyglass_big_mem',
  'workflow_manifest': '{"kind":"Workflow","apiVersion":"argoproj.io/v1alpha1","metadata":{"generateName":"shasta-spyglass-","creationTimestamp":null,"labels":{"pipelines.kubeflow.org/kfp_sdk_version":"1.2.0"},"annotations":{"pipelines.kubeflow.org/kfp_sdk_version":"1.2.0","pipelines.kubeflow.org/pi

In [94]:
#id="9d83d431-e6f1-44a2-88e2-b7cc0db18407"

arts = ['wr_17_01_59', 'wr_17_12_15', 'wr_17_23_12', 'wr_18_01_10', 'wr_18_09_59', 'wr_18_19_28', 'wr_18_29_56', 
        'wr_20_28_14', 'wr_20_36_54','wr_20_55_20', 'wr_21_09_45', 'wr_21_10_00', 'wr_21_12_53', 'wr_21_22_04', 'wr_21_31_01']

# spyglass_wf.submit(params={"s3_path": ["/path/for/vpu0-1a", "path/for/vpu0-2a" ...], "any_other": "parameters"})

#spyglass_workflow.submit(job_name='my_sg_workflow', params = {'s3_bucket': log_bucket, 's3_key': log_key})
spyglass_wf = KubeflowPipeline.retrieve(name='Spyglass_big_mem')
spyglass_wf.submit(job_name='wr_21_31_01_B', params={"s3_path": hr_2})

{'created_at': 'Thu, 17 Jun 2021 20:13:22 GMT',
 'description': None,
 'error': None,
 'finished_at': 'Thu, 01 Jan 1970 00:00:00 GMT',
 'id': 'f05bc3dc-a6a8-4afe-8240-f795c0288bf6',
 'metrics': None,
 'name': 'wr_21_31_01_B',
 'pipeline_spec': {'parameters': [{'name': 's3_path',
    'value': '["s3://brt-dcm-data/db2/vpu0-10a/full/2021-05-27_21-31-01.504959708_414_5137.620611719", "s3://brt-dcm-data/db2/vpu0-11a/full/2021-05-27_21-31-01.514079950_396_5137.853894655", "s3://brt-dcm-data/db2/vpu0-12a/full/2021-05-27_21-31-01.516012290_398_5137.836188220", "s3://brt-dcm-data/db2/vpu0-13a/full/2021-05-27_21-31-01.506979740_398_5137.745388186", "s3://brt-dcm-data/db2/vpu0-14a/full/2021-05-27_21-31-01.508356567_364_5137.919724052"]'}],
  'pipeline_id': '38144d2c-baaa-491d-9434-224572b61fb8',
  'pipeline_manifest': None,
  'pipeline_name': 'Spyglass_big_mem',
  'workflow_manifest': '{"kind":"Workflow","apiVersion":"argoproj.io/v1alpha1","metadata":{"generateName":"shasta-spyglass-","creationTi

In [ ]:
def submit_multivpu_spyglass_kubeflow(s3_paths):
    """
    Wrapper around the brtdevkit spyglass workflow object that is error tolerant.
    :param bucket: S3 bucket name
    :param s3_key: s3 key of object to be processed
    :param retry: number of times to retry the worflow submission in case of failure
    :return: tuple of workflow robot_id and s3_key
    """
    for i in range(retry):
        try:
            wf = Workflow.retrieve("Spyglass")
            result = wf.submit(params={"s3_paths": s3_paths})
        except Exception as e:
            print(f'error encountered,retrying...')
            print(e)
            time.sleep(5)
            continue
        break

    print(f"launched spyglass workflow {result['metadata']['name']}")
    return result['metadata']['name']



In [ ]:

Spyglass

# Submit the original log for evaluation
spyglass_workflow.submit(job_name='my_sg_workflow', params = {'s3_bucket': log_bucket, 's3_key': log_key})

In [2]:
spyglass_keys = ['data-flatbuffer/machine-paladin/paladin-ssd209-09032020/vpu0-9a/2020-08-20-15-35-58',
                 'data-flatbuffer/half-mule/mule-08142020/vpu0-2a/2020-08-14-17-44-19',
                 'data-flatbuffer/machine-paladin/paladin-ssd201-07182020/paladin_logs/vpu0-3a/2020-07-14-22-08-33',
                 'data-flatbuffer/machine-bradley/bradley-upload-07292020/vpu0-7a/2020-07-29-22-57-01',
                 'data-flatbuffer/duckduck/duckduck-ssd201-09082020/july11/vpu0/2020-07-10-19-39-20',
                 'data-flatbuffer/duckduck/duckduck-ssd201-09082020/shasta/full/2020-06-15-20-04-25',
                'data-flatbuffer/machine-paladin/paladin-ssd201-07182020/paladin_logs/vpu0-0a/2020-07-14-22-45-21',
                 'data-flatbuffer/duckduck/duckduck-ssd201-09082020/july11/vpu0/2020-07-10-19-48-41'
                ]
    
for i in range(len(spyglass_keys)):
    submit_spyglass_workflow(bucket = 'brt-dcm-data', s3_key = spyglass_keys[i])

launched spyglass workflow spyglass-workflow-frrf9 on key data-flatbuffer/machine-paladin/paladin-ssd209-09032020/vpu0-9a/2020-08-20-15-35-58
launched spyglass workflow spyglass-workflow-7lvf2 on key data-flatbuffer/half-mule/mule-08142020/vpu0-2a/2020-08-14-17-44-19
launched spyglass workflow spyglass-workflow-s5bj9 on key data-flatbuffer/machine-paladin/paladin-ssd201-07182020/paladin_logs/vpu0-3a/2020-07-14-22-08-33
launched spyglass workflow spyglass-workflow-kscg2 on key data-flatbuffer/machine-bradley/bradley-upload-07292020/vpu0-7a/2020-07-29-22-57-01
launched spyglass workflow spyglass-workflow-bkrhw on key data-flatbuffer/duckduck/duckduck-ssd201-09082020/july11/vpu0/2020-07-10-19-39-20
launched spyglass workflow spyglass-workflow-vll25 on key data-flatbuffer/duckduck/duckduck-ssd201-09082020/shasta/full/2020-06-15-20-04-25
launched spyglass workflow spyglass-workflow-skjgc on key data-flatbuffer/machine-paladin/paladin-ssd201-07182020/paladin_logs/vpu0-0a/2020-07-14-22-45-21


In [2]:
# Check triggers and kick off update workflows if trigger(s) are met
from brtdevkit.contrib.util.spyglass import check_spyglass_trigger

analyses = ["5f6e72bf0292f62597fe7110", '5f3dd88ff85bdd4c3c0d75ff', '5f1faea96b9d3ac8d7271ac6',
            '5f29a717ba03d3191651720b', '5f580e7ea5371bb6fac87e04', '5f58f5b65fa1a72a25d10887',
            '5f1fe43c87bbe4d5600c9769', '5f580d947f0cdf5f4b0b5848']

for a in analyses:
    
    check_spyglass_trigger(a)

2021-04-27 14:20:56,702 - Spyglass - INFO - 100 annotated images found. Threshold is 147
2021-04-27 14:20:58,319 - Spyglass - INFO - 52 annotated images found. Threshold is 26
2021-04-27 14:20:59,242 - Spyglass - INFO - Submitting Spyglass update workflow...
2021-04-27 14:20:59,951 - Spyglass - INFO - Updating triggers...
2021-04-27 14:21:01,634 - Spyglass - INFO - 52 annotated images found. Threshold is 26
2021-04-27 14:21:01,668 - Spyglass - WARNING - Multiple trigger conditions met for 5f3dd88ff85bdd4c3c0d75ff.Only sending one update job for the first trigger found.
2021-04-27 14:21:01,669 - Spyglass - INFO - Updating triggers...
2021-04-27 14:21:03,425 - Spyglass - INFO - 30 annotated images found. Threshold is 15
2021-04-27 14:21:04,157 - Spyglass - INFO - Submitting Spyglass update workflow...
2021-04-27 14:21:05,116 - Spyglass - INFO - Updating triggers...
2021-04-27 14:21:06,564 - Spyglass - INFO - 30 annotated images found. Threshold is 15
2021-04-27 14:21:06,602 - Spyglass - 